In [31]:
import os

In [32]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\aarav\\Downloads\\Second-Hand-Car-Price-Prediction-Using-MLops'

In [5]:
# Entity

In [33]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str 
    fit_intercept: bool             
    copy_X: bool 
    positive: bool
    target_column: str  

In [23]:
# Configuration Manager

In [34]:
from CarPrediction.constants import *
from CarPrediction.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer 
          params = self.params.LinearRegression 
          schema = self.schema.TARGET_COLUMN
          
          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir=config.root_dir,
                train_data_path=config.train_data_path,
                test_data_path=config.test_data_path,
                model_name= config.model_name,
                fit_intercept = params.fit_intercept,
                copy_X = params.copy_X,
                positive =  params.positive,
                target_column= schema.name
          )

          return model_trainer_config

In [26]:
# Components

In [36]:
import pandas as pd
import os 
from CarPrediction import logger 
from sklearn.linear_model import LinearRegression
import joblib 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [37]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target variable
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        # Define a column transformer for preprocessing
        numeric_features = train_x.select_dtypes(include=['int32']).columns
        numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

        categorical_features = train_x.select_dtypes(include=['object']).columns
        categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        # Combine preprocessing with the model
        model = Pipeline(steps=[('preprocessor', preprocessor),
                                ('model', LinearRegression(fit_intercept=self.config.fit_intercept, copy_X =self.config.copy_X,positive=self.config.positive))])

        # Fit the model
        model.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [29]:
# Pipeline

In [38]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-09 16:01:29,809: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-09 16:01:29,817: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-09 16:01:29,820: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-09 16:01:29,821: INFO: common: created directory at: artifacts]
[2024-01-09 16:01:29,823: INFO: common: created directory at: artifacts/model_trainer]
